In [1]:
# KAGGLE ENVIRONMENT HEADER

import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_5min.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ETH_1min.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1min.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ETH_5min.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ADA_5min.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ETH_1sec.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ADA_1sec.csv
/kaggle/input/high-frequency-crypto-limit-order-book-data/ADA_1min.csv


In [2]:
# MAIN EXPERIMENT CONFIG

COINS = ["BTC", "ETH", "ADA"]
PATHS = {
    "BTC": "/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1min.csv",
    "ETH": "/kaggle/input/high-frequency-crypto-limit-order-book-data/ETH_1min.csv",
    "ADA": "/kaggle/input/high-frequency-crypto-limit-order-book-data/ADA_1min.csv",
}
LAGS = [1, 2, 3, 5, 10, 20, 30]   # LIST OF LAGS
WINDOW = 30                # TRAIN WINDOW SIZE
MAX_LEVELS = 15            # DEPTH OF ORDER BOOK

# LOADING DATA, OFI CALCULATION

def load_coin_minute(paths, coin: str, max_levels: int = MAX_LEVELS) -> tuple[pd.DataFrame, int]:
    if isinstance(paths, str):
        df = pd.read_csv(paths)
    else:
        df = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)
        
    df["time"] = pd.to_datetime(df["system_time"], utc=True, errors="coerce")
    df = df.dropna(subset=["time"]).sort_values("time")
    df["minute"] = df["time"].dt.floor("T")
    df = df.groupby("minute", as_index=True).last()

    def has_level(k):
        need = [
            f"bids_limit_notional_{k}", f"bids_cancel_notional_{k}", f"bids_market_notional_{k}",
            f"asks_limit_notional_{k}", f"asks_cancel_notional_{k}", f"asks_market_notional_{k}",
        ]
        return all(c in df.columns for c in need)
    levels = 0
    for k in range(max_levels):
        if has_level(k):
            levels = k + 1
        else:
            break
    if levels == 0:
        raise ValueError(f"{coin}: no complete OFI levels found.")

    for k in range(levels):
        ofi_k = (
            df[f"bids_limit_notional_{k}"].fillna(0)
            - df[f"bids_cancel_notional_{k}"].fillna(0)
            - df[f"bids_market_notional_{k}"].fillna(0)
            + df[f"asks_market_notional_{k}"].fillna(0)
            + df[f"asks_cancel_notional_{k}"].fillna(0)
            - df[f"asks_limit_notional_{k}"].fillna(0)
        ).astype(float)
        df[f"{coin}__ofi_{k}"] = ofi_k
        col = f"{coin}__ofi_{k}"
        df[col] = np.sign(df[col]) * np.log1p(np.abs(df[col]))

    df[f"{coin}__ofi_0"] = df[f"{coin}__ofi_{0}"]

    df["midpoint"] = df["midpoint"].astype(float)
    logmid = np.log(df["midpoint"])
    df[f"{coin}__ret_back"] = logmid - logmid.shift(1)
    df[f"{coin}__ret_fwd"]  = logmid.shift(-1) - logmid

    keep = [c for c in df.columns if c.startswith(f"{coin}__")]
    return df[keep], levels

coin_dfs = {}
levels_by_coin = {}
for c in COINS:
    dfi, levs = load_coin_minute(PATHS[c], c, MAX_LEVELS)
    coin_dfs[c] = dfi
    levels_by_coin[c] = levs
    print(f'Loaded coin {c}, table shape: {dfi.shape}')

panel = pd.concat([coin_dfs[c] for c in COINS], axis=1, join="inner").sort_index()

# LAGGED FEATURES

for c in COINS:
    for L in LAGS:
        panel[f"{c}__ofi0_lag{L}"] = panel[f"{c}__ofi_0"].shift(L - 1)
        panel[f"{c}__ret_lag{L}"]  = panel[f"{c}__ret_back"].shift(L - 1)

for c in COINS:
    kmax = levels_by_coin[c]
    for k in range(kmax):
        base_col = f"{c}__ofi_{k}"
        for L in LAGS:
            panel[f"{base_col}_lag{L}"] = panel[base_col].shift(L - 1)

panel = panel.dropna(how="any").copy()
print(f'Concatenated, added lags, dropped NaN, table shape: {panel.shape}\n')
print(panel.head())

Loaded coin BTC, table shape: (17113, 17)
Loaded coin ETH, table shape: (17110, 17)
Loaded coin ADA, table shape: (17109, 17)
Concatenated, added lags, dropped NaN, table shape: (17073, 408)

                           BTC__ofi_0  BTC__ofi_1  BTC__ofi_2  BTC__ofi_3  \
minute                                                                      
2021-04-07 12:03:00+00:00  -13.628070   10.968163  -11.504435    8.983297   
2021-04-07 12:04:00+00:00   11.289577    9.795062  -11.632652    8.517393   
2021-04-07 12:05:00+00:00  -11.350863   -7.080649  -11.578936   -8.557264   
2021-04-07 12:06:00+00:00  -11.619749    4.912066    6.791683   -5.315224   
2021-04-07 12:07:00+00:00    7.616195   10.920076  -11.552729    8.503879   

                           BTC__ofi_4  BTC__ofi_5  BTC__ofi_6  BTC__ofi_7  \
minute                                                                      
2021-04-07 12:03:00+00:00    5.797485    8.344788    9.658293   10.075495   
2021-04-07 12:04:00+00:00   10.049945

In [3]:
# FEATURE SET FOR EACH CONFIGURATION (INCLUDING ROLLING PCA)

def build_features_train_test(t_idx: int, target_coin: str, cfg: str):
    # Train window: [t_idx-WINDOW, t_idx)
    # Test: [t_idx]
    
    rows = panel.index
    if t_idx < WINDOW:
        return None

    train_idx = rows[t_idx - WINDOW:t_idx]
    test_idx  = rows[t_idx:t_idx+1]

    y_train = panel.loc[train_idx, f"{target_coin}__ret_fwd"].values
    y_test  = float(panel.loc[test_idx, f"{target_coin}__ret_fwd"].values[0])

    cols = []

    if cfg == "ofi_best_own":
        cols += [f"{target_coin}__ofi0_lag{L}" for L in LAGS]

    elif cfg == "ofi_best_cross":
        for c in COINS:
            cols += [f"{c}__ofi0_lag{L}" for L in LAGS]

    elif cfg == "ret_own":
        cols += [f"{target_coin}__ret_lag{L}" for L in LAGS]

    elif cfg == "ret_cross":
        for c in COINS:
            cols += [f"{c}__ret_lag{L}" for L in LAGS]

    elif cfg in (
        "ofi_int_own", "ofi_int_cross",
        "ret_own_intcross_ofi", "ret_cross_intcross_ofi",
        "ret_own_intown_ofi", "ret_cross_intown_ofi",  # NEW
    ):
        if cfg in ("ret_own_intcross_ofi", "ret_own_intown_ofi"):
            cols += [f"{target_coin}__ret_lag{L}" for L in LAGS]
        elif cfg in ("ret_cross_intcross_ofi", "ret_cross_intown_ofi"):
            for c in COINS:
                cols += [f"{c}__ret_lag{L}" for L in LAGS]

        if cfg in ("ofi_int_own", "ret_own_intown_ofi", "ret_cross_intown_ofi"):
            ofi_coins = [target_coin]
        else:
            ofi_coins = COINS

        X_train_list, X_test_list = [], []

        if cols:
            X_train_base = panel.loc[train_idx, cols].values
            X_test_base  = panel.loc[test_idx,  cols].values
            if np.isnan(X_train_base).any() or np.isnan(X_test_base).any():
                return None
            X_train_list.append(X_train_base)
            X_test_list.append(X_test_base)

        for c in ofi_coins:
            kmax = levels_by_coin[c]
            for L in LAGS:
                lvl_cols = [f"{c}__ofi_{k}_lag{L}" for k in range(0, kmax)]
                Xl_train = panel.loc[train_idx, lvl_cols].values
                Xl_test  = panel.loc[test_idx,  lvl_cols].values

                if np.isnan(Xl_train).any() or np.isnan(Xl_test).any():
                    return None

                scaler = StandardScaler(with_mean=True, with_std=True)
                Xl_train_z = scaler.fit_transform(Xl_train)
                Xl_test_z  = scaler.transform(Xl_test)

                pca = PCA(n_components=1, random_state=0)
                pc_train = pca.fit_transform(Xl_train_z)[:, 0].reshape(-1, 1)
                pc_test  = pca.transform(Xl_test_z)[:, 0].reshape(1, 1)

                X_train_list.append(pc_train)
                X_test_list.append(pc_test)

        X_train = np.hstack(X_train_list)
        X_test  = np.hstack(X_test_list)
        return X_train, y_train, X_test, y_test

    else:
        raise ValueError(f"Unknown config: {cfg}")

    X_train = panel.loc[train_idx, cols].values
    X_test  = panel.loc[test_idx,  cols].values
    if np.isnan(X_train).any() or np.isnan(X_test).any():
        return None
    return X_train, y_train, X_test, y_test

In [4]:
# CONFIGS DESCRIPTION

CONFIGS = [
    "ofi_best_own",
    "ofi_int_own",
    "ret_own",
    "ofi_best_cross",
    "ofi_int_cross",
    "ret_cross",
    "ret_cross_intcross_ofi"
]

CONFIGS_OWN = [
    "ofi_best_own",
    "ofi_int_own",
    "ret_own"
]

CONFIGS_CROSS = [
    "ofi_best_cross",
    "ofi_int_cross",
    "ret_cross",
]

CONFIGS_HYBRID = [
    "ret_cross_intcross_ofi"
]


results = {cfg: {} for cfg in CONFIGS}
in_sample = {cfg: {} for cfg in CONFIGS}

start_idx = max(LAGS) + WINDOW

pred_series = {cfg: {} for cfg in CONFIGS}
true_series = {cfg: {} for cfg in CONFIGS}

In [5]:
# SINGLE-COIN MODELS

for target in COINS:
    for cfg in CONFIGS_OWN:
        print(target, cfg)
        y_true, y_pred, t_stamps = [], [], []
        train_r2 = []
        model = LinearRegression()

        rows = panel.index

        for t_idx in range(start_idx, len(panel.index)):
            built = build_features_train_test(t_idx, target, cfg)
            if built is None:
                continue
            X_train, y_train, X_test, y_test = built

            scaler_final = StandardScaler(with_mean=True, with_std=True)
            X_train_z = scaler_final.fit_transform(X_train)
            X_test_z  = scaler_final.transform(X_test)

            model.fit(X_train_z, y_train)
            y_hat = float(model.predict(X_test_z)[0])
            train_r2.append(model.score(X_train_z, y_train))

            y_true.append(y_test)
            y_pred.append(y_hat)
            t_stamps.append(rows[t_idx])

        if len(y_true) == 0:
            r2 = float("nan"); rmse = float("nan"); in_sample[cfg][target] = float("nan")
        else:
            y_true = np.array(y_true); y_pred = np.array(y_pred)
            r2 = r2_score(y_true, y_pred)
            rmse = mean_squared_error(y_true, y_pred, squared=False)
            print(r2, rmse, np.nanmean(train_r2))
            in_sample[cfg][target] = np.nanmean(train_r2)

        results[cfg][target] = {"R2": r2, "RMSE": rmse}

        if len(t_stamps):
            s_pred = pd.Series(y_pred, index=pd.to_datetime(t_stamps))
            s_true = pd.Series(y_true, index=pd.to_datetime(t_stamps))
            pred_series[cfg][target] = s_pred
            true_series[cfg][target] = s_true

BTC ofi_best_own
-0.40382672620953786 0.0012596279485901317 0.25386373497392783
BTC ofi_int_own
-0.2328832767471667 0.0011804470627701904 0.2505651992479947
BTC ret_own
-0.7432821623631742 0.0014036840925254657 0.2322050011338787
ETH ofi_best_own
-0.365269168293223 0.0015622906875794433 0.24524453124768292
ETH ofi_int_own
-0.20617647533495287 0.0014684465040598036 0.24837865655521657
ETH ret_own
-0.5028428668033704 0.001639115246022969 0.23511469194990472
ADA ofi_best_own
-0.37591262797615377 0.002418520127625504 0.2454358008681275
ADA ofi_int_own
-0.17998543163734237 0.0022397141846302294 0.24292519949405753
ADA ret_own
-0.5377004857142687 0.002556761120388564 0.23402243379930865


In [6]:
# CROSS MODELS

for target in COINS:
    for cfg in CONFIGS_CROSS:
        print(target, cfg)
        y_true, y_pred, t_stamps = [], [], []
        train_r2 = []
        model = Lasso(4e-4)

        rows = panel.index

        for t_idx in range(start_idx, len(panel.index)):
            built = build_features_train_test(t_idx, target, cfg)
            if built is None:
                continue
            X_train, y_train, X_test, y_test = built

            scaler_final = StandardScaler(with_mean=True, with_std=True)
            X_train_z = scaler_final.fit_transform(X_train)
            X_test_z  = scaler_final.transform(X_test)

            model.fit(X_train_z, y_train)
            y_hat = float(model.predict(X_test_z)[0])
            train_r2.append(model.score(X_train_z, y_train))

            y_true.append(y_test)
            y_pred.append(y_hat)
            t_stamps.append(rows[t_idx])

        if len(y_true) == 0:
            r2 = float("nan"); rmse = float("nan"); in_sample[cfg][target] = float("nan")
        else:
            y_true = np.array(y_true); y_pred = np.array(y_pred)
            r2 = r2_score(y_true, y_pred)
            rmse = mean_squared_error(y_true, y_pred, squared=False)
            print(r2, rmse, np.nanmean(train_r2))
            in_sample[cfg][target] = np.nanmean(train_r2)

        results[cfg][target] = {"R2": r2, "RMSE": rmse}

        if len(t_stamps):
            s_pred = pd.Series(y_pred, index=pd.to_datetime(t_stamps))
            s_true = pd.Series(y_true, index=pd.to_datetime(t_stamps))
            pred_series[cfg][target] = s_pred
            true_series[cfg][target] = s_true

BTC ofi_best_cross
-0.2914999515978085 0.00120818303212876 0.026346931699260945
BTC ofi_int_cross
-0.19280948823810773 0.0011611038833330334 0.024692106076033428
BTC ret_cross
-0.3815411252301848 0.0012495897199490916 0.03502014919096842
ETH ofi_best_cross
-0.2664230257012281 0.0015046729090178147 0.06398014771508703
ETH ofi_int_cross
-0.12375711008858481 0.0014173885789418314 0.06483522915830275
ETH ret_cross
-0.24026492044455328 0.0014890522352148353 0.08544567872610102
ADA ofi_best_cross
-0.39375765775430493 0.0024341532157017795 0.1422510401565453
ADA ofi_int_cross
-0.21684665262476077 0.002274428054630979 0.13770494776728928
ADA ret_cross
-0.3138233264179058 0.0023633212606490104 0.12177911020428366


In [7]:
# HYBRID MODEL

for target in COINS:
    for cfg in CONFIGS_HYBRID:
        print(target, cfg)
        y_true, y_pred, t_stamps = [], [], []
        train_r2 = []
        model = Lasso(8e-4)

        rows = panel.index

        for t_idx in range(start_idx, len(panel.index)):
            built = build_features_train_test(t_idx, target, cfg)
            if built is None:
                continue
            X_train, y_train, X_test, y_test = built

            scaler_final = StandardScaler(with_mean=True, with_std=True)
            X_train_z = scaler_final.fit_transform(X_train)
            X_test_z  = scaler_final.transform(X_test)

            model.fit(X_train_z, y_train)
            y_hat = float(model.predict(X_test_z)[0])
            train_r2.append(model.score(X_train_z, y_train))

            y_true.append(y_test)
            y_pred.append(y_hat)
            t_stamps.append(rows[t_idx])  # record the test timestamp

        if len(y_true) == 0:
            r2 = float("nan"); rmse = float("nan"); in_sample[cfg][target] = float("nan")
        else:
            y_true = np.array(y_true); y_pred = np.array(y_pred)
            r2 = r2_score(y_true, y_pred)
            rmse = mean_squared_error(y_true, y_pred, squared=False)
            print(r2, rmse, np.nanmean(train_r2))
            in_sample[cfg][target] = np.nanmean(train_r2)

        results[cfg][target] = {"R2": r2, "RMSE": rmse}

        if len(t_stamps):
            s_pred = pd.Series(y_pred, index=pd.to_datetime(t_stamps))
            s_true = pd.Series(y_true, index=pd.to_datetime(t_stamps))
            pred_series[cfg][target] = s_pred
            true_series[cfg][target] = s_true

BTC ret_cross_intcross_ofi
-0.1717183751916822 0.001150792851254763 0.010366624085050131
ETH ret_cross_intcross_ofi
-0.10041676325826265 0.0014025918192351748 0.03279277966155628
ADA ret_cross_intcross_ofi
-0.11659640006581751 0.0021787248628131707 0.04934155426256571


In [8]:
# IN-SAMPLE METRICS

print("Metrics (R2, RMSE) per coin and averaged:")
rows = []
for cfg in CONFIGS:
    in_sample_list = []
    for c in COINS:
        print(f"{cfg:25s} | {c:4s} | R2={in_sample[cfg][c]: .4f}")
        in_sample_list.append(in_sample[cfg][c])
    
    print(f"{cfg:25s} | AVG  | R2={np.nanmean(in_sample_list): .4f}")
    print("-" * 70)

summary = pd.DataFrame(rows, columns=["config", "R2_avg", "RMSE_avg"]).set_index("config")
print("\nSummary:\n", summary)

Metrics (R2, RMSE) per coin and averaged:
ofi_best_own              | BTC  | R2= 0.2539
ofi_best_own              | ETH  | R2= 0.2452
ofi_best_own              | ADA  | R2= 0.2454
ofi_best_own              | AVG  | R2= 0.2482
----------------------------------------------------------------------
ofi_int_own               | BTC  | R2= 0.2506
ofi_int_own               | ETH  | R2= 0.2484
ofi_int_own               | ADA  | R2= 0.2429
ofi_int_own               | AVG  | R2= 0.2473
----------------------------------------------------------------------
ret_own                   | BTC  | R2= 0.2322
ret_own                   | ETH  | R2= 0.2351
ret_own                   | ADA  | R2= 0.2340
ret_own                   | AVG  | R2= 0.2338
----------------------------------------------------------------------
ofi_best_cross            | BTC  | R2= 0.0263
ofi_best_cross            | ETH  | R2= 0.0640
ofi_best_cross            | ADA  | R2= 0.1423
ofi_best_cross            | AVG  | R2= 0.0775
---------

In [9]:
# OUT-OF-SAMPLE METRICS

print("Metrics (R2, RMSE) per coin and averaged:")
rows = []
for cfg in CONFIGS:
    r2_list = []
    rmse_list = []
    for c in COINS:
        r2 = results[cfg][c]["R2"]
        rmse = results[cfg][c]["RMSE"]
        r2_list.append(r2)
        rmse_list.append(rmse)
        print(f"{cfg:25s} | {c:4s} | R2={r2: .4f} | RMSE={rmse: .6f}")
    r2_avg = np.nanmean(r2_list)
    rmse_avg = np.nanmean(rmse_list)
    rows.append((cfg, r2_avg, rmse_avg))
    print(f"{cfg:25s} | AVG  | R2={r2_avg: .4f} | RMSE={rmse_avg: .6f}")
    print("-" * 70)

summary = pd.DataFrame(rows, columns=["config", "R2_avg", "RMSE_avg"]).set_index("config")
print("\nSummary:\n", summary)

Metrics (R2, RMSE) per coin and averaged:
ofi_best_own              | BTC  | R2=-0.4038 | RMSE= 0.001260
ofi_best_own              | ETH  | R2=-0.3653 | RMSE= 0.001562
ofi_best_own              | ADA  | R2=-0.3759 | RMSE= 0.002419
ofi_best_own              | AVG  | R2=-0.3817 | RMSE= 0.001747
----------------------------------------------------------------------
ofi_int_own               | BTC  | R2=-0.2329 | RMSE= 0.001180
ofi_int_own               | ETH  | R2=-0.2062 | RMSE= 0.001468
ofi_int_own               | ADA  | R2=-0.1800 | RMSE= 0.002240
ofi_int_own               | AVG  | R2=-0.2063 | RMSE= 0.001630
----------------------------------------------------------------------
ret_own                   | BTC  | R2=-0.7433 | RMSE= 0.001404
ret_own                   | ETH  | R2=-0.5028 | RMSE= 0.001639
ret_own                   | ADA  | R2=-0.5377 | RMSE= 0.002557
ret_own                   | AVG  | R2=-0.5946 | RMSE= 0.001867
------------------------------------------------------------

In [10]:
# PORTFOLIO GENERATION

def portfolio_from_preds(pred_series_cfg: dict, true_series_cfg: dict, coins=COINS):
    union_idx = pd.Index([])
    for c in coins:
        if c in pred_series_cfg:
            union_idx = union_idx.union(pred_series_cfg[c].index)
        if c in true_series_cfg:
            union_idx = union_idx.union(true_series_cfg[c].index)
    union_idx = union_idx.sort_values()

    if len(union_idx) == 0:
        return {"N": 0, "TotLogRet": np.nan, "TotRet_mult": np.nan,
                "Mean": np.nan, "Std": np.nan, "Sharpe_min": np.nan}, \
               pd.DataFrame()

    P = np.column_stack([
        pred_series_cfg.get(c, pd.Series(0.0, index=union_idx)).reindex(union_idx).fillna(0.0).values
        for c in coins
    ])
    R = np.column_stack([
        true_series_cfg.get(c, pd.Series(0.0, index=union_idx)).reindex(union_idx).fillna(0.0).values
        for c in coins
    ])

    W_raw = np.maximum(P, 0.0)
    s = W_raw.sum(axis=1, keepdims=True)
    W = np.divide(W_raw, s, out=np.zeros_like(W_raw), where=(s > 0))  
    # no positive predictions -> do all 0

    r_port = (W * R).sum(axis=1)
    equity = np.exp(np.cumsum(r_port))

    mu = float(np.mean(r_port)) if len(r_port) else np.nan
    sd = float(np.std(r_port, ddof=1)) if len(r_port) > 1 else np.nan
    sharpe_min = (mu / sd) if (sd is not None and sd and not np.isnan(sd)) else np.nan

    metrics = {
        "N": int(len(r_port)),
        "TotLogRet": float(np.sum(r_port)) if len(r_port) else np.nan,
        "TotRet_mult": float(equity[-1]) if len(equity) else np.nan,  # final wealth multiple
        "Mean": mu,
        "Std": sd,
        "Sharpe_min": sharpe_min,
    }
    port_df = pd.DataFrame({"r_port": r_port, "equity": equity}, index=union_idx)
    return metrics, port_df

In [11]:
# ECONOMIC UTILITY METRICS 

portfolio_metrics = {}
portfolio_paths = {}

for cfg in CONFIGS:
    m, path = portfolio_from_preds(pred_series[cfg], true_series[cfg], coins=COINS)
    portfolio_metrics[cfg] = m
    portfolio_paths[cfg] = path
    print(cfg, " | N:", m["N"], " | Wealth x:", m["TotRet_mult"], " | Sharpe/min:", m["Sharpe_min"])


ofi_best_own  | N: 17013  | Wealth x: 1.1615141000175617  | Sharpe/min: 0.006403444350658536
ofi_int_own  | N: 17013  | Wealth x: 1.2049670310418739  | Sharpe/min: 0.008392347778790436
ret_own  | N: 17013  | Wealth x: 1.0960957984846067  | Sharpe/min: 0.004199573364453803
ofi_best_cross  | N: 17013  | Wealth x: 0.8631540470863839  | Sharpe/min: -0.006399438137617491
ofi_int_cross  | N: 17013  | Wealth x: 0.9667831920930047  | Sharpe/min: -0.0014692725223032375
ret_cross  | N: 17013  | Wealth x: 1.666129998345505  | Sharpe/min: 0.02307375713608777
ret_cross_intcross_ofi  | N: 17013  | Wealth x: 1.272327507399807  | Sharpe/min: 0.011088014993963497
